Task1

In [ ]:
defect_given_plan = {
    'P1': 0.01,
    'P2': 0.03,
    'P3': 0.02
}

plan_probabilities = {
    'P1': 0.30,
    'P2': 0.20,
    'P3': 0.50
}

def calculate_probability(defect_given_plan, plan_probabilities):
    total_defect_probability = sum(defect_given_plan[plan] * plan_probabilities[plan] for plan in defect_given_plan)

    probabilities = {}
    for plan in defect_given_plan:
        probabilities[plan] = (defect_given_plan[plan] * plan_probabilities[plan]) / total_defect_probability

    return probabilities

def find_responsible_plan(probabilities):
    return max(probabilities, key=probabilities.get)

# Calculate probabilities
probabilities = calculate_probability(defect_given_plan, plan_probabilities)

# Find responsible plan
responsible_plan = find_responsible_plan(probabilities)

# Output probabilities
print("Probability of each plan being responsible for a defective product:")
for plan, probability in probabilities.items():
    print(f"Plan {plan}: {probability:.2f}")

# Output most likely plan
print(f"\nThe plan most likely responsible for the defective product is: Plan {responsible_plan}")


Probability of each plan being responsible for a defective product:
Plan P1: 0.16
Plan P2: 0.32
Plan P3: 0.53

The plan most likely responsible for the defective product is: Plan P3


Task2

In [4]:
!pip install pomegranate==v0.14.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 13.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for pomegranate: filename=pomegranate-0.14.9-cp310-cp310-linux_x86_64.whl size=18331394 sha256=eb9851d07e30dbbf44f6832eb8c7024051d8e104f1a106e5d70b6e82a5c40370
  Stored in directory: /root/.cache/pip/wheels/14/e7/b2/189a2d351ac4ae073cfa17ce9d56936d59af5712a18028fc31
Successfully built pomegranate
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [5]:
from pomegranate import *

# Define the distributions for each box
box1 = DiscreteDistribution({'gold': 2, 'silver': 0})
box2 = DiscreteDistribution({'gold': 0, 'silver': 2})
box3 = DiscreteDistribution({'gold': 1, 'silver': 1})

# Create the conditional probability tables
cpd_box1 = ConditionalProbabilityTable(
    [['gold', 'gold', 1.0],
     ['gold', 'silver', 0.0],
     ['silver', 'gold', 0.0],
     ['silver', 'silver', 1.0]],
    [box1])

cpd_box2 = ConditionalProbabilityTable(
    [['gold', 'gold', 1.0],
     ['gold', 'silver', 0.0],
     ['silver', 'gold', 0.0],
     ['silver', 'silver', 1.0]],
    [box2])

cpd_box3 = ConditionalProbabilityTable(
    [['gold', 'gold', 0.5],
     ['gold', 'silver', 0.5],
     ['silver', 'gold', 0.5],
     ['silver', 'silver', 0.5]],
    [box3])

# Create the states for each box
s1 = State(box1, name="Box 1")
s2 = State(box2, name="Box 2")
s3 = State(box3, name="Box 3")

# Create the states for each conditional probability table
s_box1 = State(cpd_box1, name="Box 1 CPD")
s_box2 = State(cpd_box2, name="Box 2 CPD")
s_box3 = State(cpd_box3, name="Box 3 CPD")

# Create the Bayesian network
model = BayesianNetwork("Coin Picking")

# Add the states and conditional probability tables to the model
model.add_states(s1, s2, s3, s_box1, s_box2, s_box3)

# Add transitions from the box states to the corresponding CPD states
model.add_edge(s1, s_box1)
model.add_edge(s2, s_box2)
model.add_edge(s3, s_box3)

# Add transitions from the CPD states to the box states
model.add_edge(s_box1, s1)
model.add_edge(s_box2, s2)
model.add_edge(s_box3, s3)

# Finalize the model
model.bake()

# Calculate the probabilities
probabilities = {}

# Calculate probability for Box 1
model.predict_proba({'Box 1': 'gold'})
probabilities['Box 1'] = model.predict_proba({'Box 1': 'gold'})[3].parameters[0]['gold']

# Calculate probability for Box 2
model.predict_proba({'Box 2': 'gold'})
probabilities['Box 2'] = model.predict_proba({'Box 2': 'gold'})[4].parameters[0]['gold']

# Calculate probability for Box 3
model.predict_proba({'Box 3': 'gold'})
probabilities['Box 3'] = model.predict_proba({'Box 3': 'gold'})[5].parameters[0]['gold']

# Output probabilities
for box, prob in probabilities.items():
    print(f"Probability that the other coin in {box} is gold given that a gold coin was picked: {prob:.2f}")


Probability that the other coin in Box 1 is gold given that a gold coin was picked: 1.00
Probability that the other coin in Box 2 is gold given that a gold coin was picked: 0.00
Probability that the other coin in Box 3 is gold given that a gold coin was picked: 0.50
